### ENV Test

In [6]:
import sys

import tensorflow.keras
import sklearn as sk
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-13.1-arm64-arm-64bit
Tensor Flow Version: 2.9.0
Keras Version: 2.9.0

Python 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:25:29) [Clang 14.0.6 ]
Scikit-Learn 1.2.0
GPU is available


In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.layers import TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from tensorflow.keras.utils import to_categorical
from keras.layers import MaxPooling2D, Conv2D
import numpy as np
import os
from tensorflow.python.keras.callbacks import EarlyStopping
from datetime import datetime
from keras import regularizers


##  Load frames

### Old dataset

In [6]:
drowsines_levels = np.array(["0", "5", "10"])
DATA_PATH = os.path.join('./Dataset')
label_map = {label: num for num, label in enumerate(drowsines_levels)}

files, labels = [], []

for video_index in range(1,11):
  for level in drowsines_levels:
    window = []      
    for frame in range(1, 4001, 2):
      file_path = os.path.join(DATA_PATH, str(video_index), level, (str(frame) + ".npy"))
      if os.path.exists(file_path):
        res = np.load(file_path)        
        window.append(res)
      else:
        print(f"File at {file_path} doesn't exists")
    files.append(window)
    labels.append(label_map[level])
  print(f"File number: {video_index} processed")

File number: 1 processed
File number: 2 processed
File number: 3 processed
File number: 4 processed
File number: 5 processed
File number: 6 processed
File number: 7 processed
File number: 8 processed
File number: 9 processed
File number: 10 processed


In [7]:
# Check loaded files shape
np.array(files).shape

(30, 2000, 1434)

### New dataset


In [16]:
drowsines_levels = np.array(["Sleepy", "NonSleepy"])
DATA_PATH = os.path.join('./Extracted_Values_Bigger_Confidence')
label_map = {label: num for num, label in enumerate(drowsines_levels)}

files, labels = [], []

for level in drowsines_levels:
    for video_index in range(1,90):
        window = []      
        for frame in range(1, 1001):
            file_path = os.path.join(DATA_PATH, level, str(video_index), (str(frame) + ".npy"))
            if os.path.exists(file_path):
                res = np.load(file_path)        
                window.append(res)
            else:
                print(f"File at {file_path} doesn't exists")
        files.append(window)
        labels.append(label_map[level])
        print(f"File number: {video_index} processed for level {level}")

File number: 1 processed for level Sleepy
File number: 2 processed for level Sleepy
File number: 3 processed for level Sleepy
File number: 4 processed for level Sleepy
File number: 5 processed for level Sleepy
File number: 6 processed for level Sleepy
File number: 7 processed for level Sleepy
File number: 8 processed for level Sleepy
File number: 9 processed for level Sleepy
File number: 10 processed for level Sleepy
File number: 11 processed for level Sleepy
File number: 12 processed for level Sleepy
File number: 13 processed for level Sleepy
File number: 14 processed for level Sleepy
File number: 15 processed for level Sleepy
File number: 16 processed for level Sleepy
File number: 17 processed for level Sleepy
File number: 18 processed for level Sleepy
File number: 19 processed for level Sleepy
File number: 20 processed for level Sleepy
File number: 21 processed for level Sleepy
File number: 22 processed for level Sleepy
File number: 23 processed for level Sleepy
File number: 24 proc

In [12]:
np.array(files).shape

(178, 999, 1434)

## Model training

In [23]:
now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

log_dir = os.path.join(f'Logs_{date_string}')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1000, 1434)))
model.add(LSTM(128, return_sequences=True, activation='tanh')) 
model.add(LSTM(32, return_sequences=False, activation='tanh')) 
model.add(Dense(32, activation='softmax', kernel_regularizer='l2')) 
model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

model.summary()
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=500, batch_size=50, shuffle = True, callbacks=[tb_callback], validation_split = 0.2) 

model.save(f'drowsines_levels_base_good_run_weights_${date_string}.h5')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 1000, 64)          383744    
                                                                 
 lstm_19 (LSTM)              (None, 1000, 128)         98816     
                                                                 
 lstm_20 (LSTM)              (None, 32)                20608     
                                                                 
 dense_12 (Dense)            (None, 32)                1056      
                                                                 
 dense_13 (Dense)            (None, 2)                 66        
                                                                 
Total params: 504,290
Trainable params: 504,290
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500


2023-04-16 00:54:23.215217: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-16 00:54:23.775416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-16 00:54:26.383610: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-16 00:54:29.305683: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-16 00:54:32.087230: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-16 00:54:39.889477: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-16 00:54:47.715012: I tensorflow/core/grappler/optimizers/cust

3/3 [==============================] - ETA: 0s - loss: 1.0156 - accuracy: 0.4867

2023-04-16 00:55:28.948274: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-16 00:55:29.162005: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-16 00:55:31.672899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-16 00:55:34.178728: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 75s 20s/step - loss: 1.0156 - accuracy: 0.4867 - val_loss: 1.0060 - val_accuracy: 0.5172
Epoch 2/500
3/3 [==============================] - 4s 1s/step - loss: 1.0051 - accuracy: 0.4867 - val_loss: 0.9980 - val_accuracy: 0.5172
Epoch 3/500
3/3 [==============================] - 3s 1s/step - loss: 0.9972 - accuracy: 0.4602 - val_loss: 0.9897 - val_accuracy: 0.3793
Epoch 4/500
3/3 [==============================] - 3s 1s/step - loss: 0.9872 - accuracy: 0.4867 - val_loss: 0.9801 - val_accuracy: 0.5172
Epoch 5/500
3/3 [==============================] - 3s 1s/step - loss: 0.9789 - accuracy: 0.4867 - val_loss: 0.9706 - val_accuracy: 0.5172
Epoch 6/500
3/3 [==============================] - 3s 1s/step - loss: 0.9695 - accuracy: 0.4867 - val_loss: 0.9609 - val_accuracy: 0.5172
Epoch 7/500
3/3 [==============================] - 3s 1s/step - loss: 0.9605 - accuracy: 0.4867 - val_loss: 0.9517 - val_accuracy: 0.5172
Epoch 8/500
3/3 [===========================

In [21]:
now = datetime.now()
date_string = now.strftime("%d/%m/%Y %H:%M:%S")

X = np.array(files)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# es = EarlyStopping(monitor='val_loss', patience=50)

log_dir = os.path.join(f'Logs_15')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(1000, 1434)))
model.add(LSTM(128, return_sequences=True, activation='tanh')) 
model.add(TimeDistributed(Dropout(0.1)))
model.add(LSTM(32, return_sequences=False, activation='tanh')) 
model.add(Dense(32, activation='softmax', kernel_regularizer=regularizers.l2(0.008))) 
model.add(Dense(drowsines_levels.shape[0], activation='softmax'))

model.summary()
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=500, batch_size=50, shuffle = True, callbacks=[tb_callback], validation_split = 0.2) 

model.save(f'drowsines_weights_noStop_l2008_drop01_${date_string}.h5')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 1000, 64)          383744    
                                                                 
 lstm_13 (LSTM)              (None, 1000, 128)         98816     
                                                                 
 time_distributed_4 (TimeDis  (None, 1000, 128)        0         
 tributed)                                                       
                                                                 
 lstm_14 (LSTM)              (None, 32)                20608     
                                                                 
 dense_8 (Dense)             (None, 32)                1056      
                                                                 
 dense_9 (Dense)             (None, 2)                 66        
                                                      

2023-04-15 23:47:41.186268: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-15 23:47:41.732215: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-15 23:47:44.462998: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-15 23:47:47.082391: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-15 23:47:49.858149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-15 23:47:57.629438: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-15 23:48:05.395223: I tensorflow/core/grappler/optimizers/cust

3/3 [==============================] - ETA: 0s - loss: 0.9495 - accuracy: 0.5133

2023-04-15 23:48:46.130917: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-15 23:48:46.352430: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-15 23:48:48.863518: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-15 23:48:51.382139: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 75s 20s/step - loss: 0.9495 - accuracy: 0.5133 - val_loss: 0.9327 - val_accuracy: 0.5862
Epoch 2/500
3/3 [==============================] - 5s 1s/step - loss: 0.9401 - accuracy: 0.5133 - val_loss: 0.9259 - val_accuracy: 0.5862
Epoch 3/500
3/3 [==============================] - 3s 1s/step - loss: 0.9324 - accuracy: 0.5133 - val_loss: 0.9161 - val_accuracy: 0.5862
Epoch 4/500
3/3 [==============================] - 3s 1s/step - loss: 0.9241 - accuracy: 0.5133 - val_loss: 0.9072 - val_accuracy: 0.5862
Epoch 5/500
3/3 [==============================] - 3s 1s/step - loss: 0.9187 - accuracy: 0.5133 - val_loss: 0.8992 - val_accuracy: 0.5862
Epoch 6/500
3/3 [==============================] - 3s 1s/step - loss: 0.9098 - accuracy: 0.5133 - val_loss: 0.8923 - val_accuracy: 0.5862
Epoch 7/500
3/3 [==============================] - 3s 1s/step - loss: 0.9038 - accuracy: 0.5133 - val_loss: 0.8857 - val_accuracy: 0.5862
Epoch 8/500
3/3 [===========================

KeyboardInterrupt: 

## Model validation

### Confusion Matrix

In [15]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)



2/2 [==============================] - 1s 517ms/step


array([[[16,  4],
        [ 1, 15]],

       [[15,  1],
        [ 4, 16]]])

### Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir Logs_15/04/2023 21:41:01